Using **Hyperparameter Selection**, **Tuning**, and **Neural Network Learning Rate**, we are able to apply a **grid search (with brute force)** to find <font color= "red"> **optimal hyperparameters** </font>. However, when the hyperparameter space is enormous, using brute force will take too much time.

Evolutionary Algorithms (EA) such as <font color= "blue"> **Genetic Algorithm** </font> have proven to be powerful. This algorithm uses **evolution**, **fitness**, **crossover**, and **mutation**. 
<br>
<br>
We start by importing all libraries as follows:

In [5]:
from functools import reduce
from operator import add
import random

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Activation, MaxPooling2D, Flatten
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

After importing the libraries, we set some of the hyperparameters:

In [6]:
n_classes = 5
batch_size = 128
n_epochs = 1000

Next, we load and preprocess the training and validation data:

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=False,
                                   validation_split=0.25)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('data',
                                                target_size = (150,150),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 subset = "training")

validation_set = train_datagen.flow_from_directory('data',
                                            target_size = (150,150),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            subset = "validation")

Found 3243 images belonging to 5 classes.
Found 1080 images belonging to 5 classes.


Next, we define a function that creates and compiles a model. 
<br><br>
Some of the settings of our network are not set, but can be set as parameter. These parameters are the **dropout percentage**, **the learning rate**, and **the number of hidden units** in our final fully connected layer:

In [12]:
def create_model(parameters, n_classes, input_shape):
    print(parameters)
    dropout = parameters['dropout']
    learning_rate = parameters['learning_rate']
    hidden_inputs = parameters['hidden_inputs']

    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(hidden_inputs))
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    opt = Adam(learning_rate)

    model.compile(loss='categorical_crossentropy', 
    optimizer=opt, metrics=['accuracy'])

    return model

We need a class Network that we can use to create a network with random parameters and train the network. 
<br><br>
Moreover, it should be able to retrieve the accuracy of the network:

In [13]:
class Network():
    def __init__(self, parameter_space=None):
        self.accuracy = 0.
        self.parameter_space = parameter_space
        self.network_parameters = {}

    def set_random_parameters(self):
        for parameter in self.parameter_space:
            self.network_parameters[parameter] = random.choice(self.parameter_space[parameter])

    def create_network(self, network):
        self.network_parameters = network

    def train(self):
        callbacks = [EarlyStopping(monitor='val_acc', patience=5)]
        model = create_model(self.network_parameters, n_classes, input_shape)
        history = model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_data=(X_val, y_val), callbacks=callbacks)
        self.accuracy = max(history.history['val_acc'])  

Next, we will be defining a class that does the heavy lifting. <br>
Our class GA should be able to create a random population and evolve–including breeding and mutating a network.<br>
Also, it should be able to retrieve some statistics of the networks in the population:<br>

In [14]:
class Genetic_Algorithm():
    def __init__(self, parameter_space, retain=0.3, random_select=0.1, mutate_prob=0.25):
        self.mutate_prob = mutate_prob
        self.random_select = random_select
        self.retain = retain
        self.parameter_space = parameter_space

    def create_population(self, count):
        population = []
        for _ in range(0, count):
            network = Network(self.parameter_space)
            network.set_random_parameters()
            population.append(network)
        return population

    def get_fitness(network):
        return network.accuracy

    def get_grade(self, population):
        total = reduce(add, (self.fitness(network) 
        for network in population))
        return float(total) / len(population)

    def breed(self, mother, father):
        children = []
        for _ in range(2):
            child = {}
            for param in self.parameter_space:
                child[param] = random.choice(
                    [mother.network[param],
                    father.network[param]]
                )
            network = Network(self.nn_param_choices)
            network.create_set(child)
            if self.mutate_chance > random.random():
                network = self.mutate(network)
            children.append(network)
        return children

    def mutate(self, network):
        mutation = random.choice(list(self.parameter_space.keys()))
        network.network[mutation] = random.choice(self.parameter_space[mutation])
        return network

    def evolve(self, pop):
        graded = [(self.fitness(network),
        network) for network in pop]
        graded = [x[1] for x in sorted(graded,
        key=lambda x: x[0], reverse=True)]
        retain_length = int(len(graded)*self.retain)

        parents = graded[:retain_length]

        for individual in graded[retain_length:]:
            if self.random_select > random.random():
                parents.append(individual)

        parents_length = len(parents)
        desired_length = len(pop) - parents_length
        children = []

        while len(children) < desired_length:

            male = random.randint(0, 
            parents_length-1)
            female = random.randint(0, 
            parents_length-1)

            if male != female:
                male = parents[male]
                female = parents[female]

                children_new = self.breed(male,
                 female)

                for child_new in children_new:
                    if len(children) < desired_length:
                        children.append(child_new)

        parents.extend(children)

        return parents

Our last function will retrieve the average accuracy across a population:

In [15]:
def get_population_accuracy(population):
    total_accuracy = 0
    for network in population:
        total_accuracy += network.get_accuracy

    return total_accuracy / len(population)

We can now set the remaining hyperparameters that we want to explore:

In [16]:
n_generations = 10
population_size = 20

parameter_space = {
    'dropout': [0.25, 0.5, 0.75],
    'hidden_inputs': [256, 512, 1024],
    'learning_rate': [0.1, 0.01, 0.001, 0.0001]
}

Next, we create our Genetic_Algorithm and create a population:

In [17]:
GA = Genetic_Algorithm(parameter_space)
population = GA.create_population(population_size)

In [ ]:
for i in range(n_generations):
    print('Generation {}'.format(i))

    for network in population:
        network.train() 

    average_accuracy = get_population_accuracy(population)
    print('Average accuracy: {:.2f}'.format(average_accuracy))

    # Evolve
    if i < n_generations - 1:
        s = GA.evolve(networks)